In [1]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline

from make_map import make_map

from file_util import load_matlab_data
from channel_overlap import channel_overlap
from plot import plot_discharge
from metrics import *
from stratigraphy import sedimentograph


import cPickle as pickle

In [2]:
TH_TOPO = -0.5
TH_ANGLE = 75
TH_VEL = 0.3 # use the threshold value for mud deposition (U_dep_mud)
SFRAC = 25 # Sand fraction (percentage)
DEPTH = 100 # Depth of synthetic strata (m)

fbase = 'data'                          # Base file name
froot = 'ForHollyMichael/'              # Folder name
dbase = 'shuffled'                      # Base shuffled strata name
droot = str(SFRAC) + 'sand/' + str(DEPTH) + 'm/'  # Root for shuffled strata
sroot = 'shuffled/'                     # Save folder root

In [3]:
steps = 31              # Number of model timesteps
synsteps = int(DEPTH/0.05)  # Number of synthetic steps to recreate
L = 85                  # Number of cells in x-direction
W = 170                 # Number of cells in y-direction
syncount = 20           # Number of synthetic strata sets

# Generate zero matricies to use for matching
topomatch = np.zeros((L, W, steps))
uwmatch = np.zeros((L, W, steps))

# Generate zero matricies to populate and save
topo = np.zeros((L, W, synsteps))
uw = np.zeros((L, W, synsteps))

In [4]:
for fnum in range(3070,3101):
    
    # Load up the model realization data
    
    fname = fbase + str(fnum)
    data = load_matlab_data(froot, fname)
    
    # Put the topo and uw data into matricies for matching
    
    topomatch[:,:,fnum-3070] = data['eta']
    uwmatch[:,:,fnum-3070] = data['uw']

In [7]:
# Load up model strata data from layers 200-500 that was used for shuffling
data = load_matlab_data(froot, fbase + '3100')
smatch = np.zeros((np.shape(data['strata'])[0],np.shape(data['strata'])[1],300))
for i in range(200,500):
    smatch[:,:,i-200] = data['strata'][:, :, i]

In [9]:
for dnum in range(3101,3121):
      
    dname = dbase + str(dnum)
    # Load in the shuffled strata
    strata = scipy.io.loadmat(droot + dname + '.mat')['synthetic']
    
    # Match up the topo and uw to shuffled strata in such that each
    # topo or uw field maps to 10 layers of strata
    for j in range(0, synsteps):
        for k in range(0, np.shape(smatch)[2]):
            if np.array_equal(strata[:,:,j],smatch[:,:,k]) == True:
                m = int(np.around(((k/10)-0.5)))
                topo[:,:,j] = topomatch[:,:,m]
                uw[:,:,j] = uwmatch[:,:,m]
    
    # Store information in dictionary format
    data = {}
    match = {}
    match['topo'] = topo
    match['uw'] = uw
    match['strata'] = strata
    data['match'] = match
    
    ## Pickling process
    
    #pickle.dump( match, open( sroot + dname + '.p', "wb" ) )
    
    # Save data as a .mat file
    
    scipy.io.savemat(sroot + dname + '.mat', data, do_compression=True)

KeyboardInterrupt: 

In [ ]:
b = scipy.io.loadmat(sroot + dbase + '3101.mat')['match']

In [ ]:
c = b['topo'][0][0]

In [ ]:
np.shape(c)

In [ ]:
np.sum(c[:,:,1999])

In [8]:
np.shape(smatch)

(85, 170, 300)

In [ ]:
501-200